In [31]:
import numpy as np
import matplotlib.pyplot as plt

In [61]:
#I defined some fun functions below to test the integral script
def func1():
    """
    Arbitrary function (sin(x)) to test the integral routine on. 
        -Inputs: x (array) values 
        -Outpus: the Integral result of sin(x)
    """
    f = lambda x: np.arctan(x)
    F = lambda x: 1.0/(1.0+x**2)
    return f,'1/(1+x^2)',F

def func2():
    """
    Arbitrary function (6x^2+12x^3) to test the integral routine on. 
        -Inputs: x (array) values 
        -Outpus: the Integral result of 6x^2+12x^3
    """
    f = lambda x: 6*x**2+ 12*x**3
    F = lambda x: 2*x**3 + 3*x**4
    return f,'6x^2+12x^3',F

def func3():
    """
    Arbitrary function (5x*e^(7x)) to test the integral routine on. 
        -Inputs: x (array) values 
        -Outpus: the Integral result of the above function
    """
    f = lambda x: 5*x*np.e**(7*x)
    F = lambda x: 5*(7*x-1)*np.e**(7*x)/49
    return f,'5xe^(7x)',F

In [75]:
#This block is a script to run simpson's rule, but on half the interval
def simpson(f,a,b,fa,fb):
    """
    Function that integrates f the interval a to b by splitting it in n subinterval and using Simpson's rule 
        -Inputs: f (function) this is the function to be integrated 
                 a (float) Lower bound of the intergral
                 b (float) Upper bound of the integral 
                 fa (float) Value of f at a
                 fb (float) Value of f at b
        -Ouputs: I (float) Value of the integral between the inteval a to b
                 h (float) Midpoint of the function
                 fh (float) Value of f at h
    """
    #Juts checks a bunch of cases for the interval a to b
    if a == b:
        return 0
    if a < b:
        sign = 1
    else:
        sign = -1
    h = (a+b)/2    #Calculates the height 
    fh = f(h)      #The value at that height
    dx = (b-a)/(6) #usually the denominator is 3*n, but here n=2, since we are halving the interval
    I = dx*sign*(fa+4*fh+fb) #--> Simpson's rule 
    return I, h, fh 

def dynamic_simpson(f,a,b,h,fa,fb,fh,guess,tolerance,nmax,count=0):
    """
    Function that estimates the error of the integral using |S(a,h)+S(h,b)-S(a,b)|<15*tolerance (this is 
    from the wikipedia page about adaptative simpson's rule), where [a,b] is our integral, and S is 
    the value of simspon rule. 
    If this error is smaller than the tolerance, it gives back the normal value of the intgral using 
    simpson(). If not, it splits the interval into another half and tests the error again.
        -Inputs: f (function) Function that is integrated 
                 a (float) Lower bound of the integral 
                 b (float) Upper bound of the integral 
                 h (float) Half point of the interval 
                 fa (float) Value of f at a
                 fb (float) Value of f at b
                 fh (float) Value of f at h 
                 guess (float) Given by Simpson's rule on the whole interval 
                 tolerance (float) Tolerance on how precise the integral is
                 nmax (int) maximum number of interation before exiting the recursion
                 count (int) number of iterations that it took for the integral to converge 
    
        -Ouputs: I (float) integral value using simpson's rule
                 Iter (int) how many iterations it took for the intergal to converge using the recursive
                 algorithm below. 
    """
    if count < nmax:
        Guess_ah,l_h,l_fh = simpson(f,a,h,fa,fh)
        Guess_hb,r_h, r_fh = simpson(f,h,b,fh,fb)
        error = np.abs(Guess_ah+Guess_hb - guess)
        if error < 15*tolerance:
            return guess, count
        else:
            guess_l, count_l = dynamic_simpson(f,a,h,l_h,fa,fh,l_fh,Guess_ah,tolerance/2,nmax,count+1)
            guess_r, count_r = dynamic_simpson(f,h,b,r_h,fh,fb,r_fh,Guess_hb,tolerance/2,nmax, count+1)
            
            guess_t = guess_l+guess_r
            count_t = count_l+count_r
            return guess_t,count_t
    else:
        print(f'Sadly my integral did not converge in {nmax} steps...oh well :(. Maybe increase nmax or another function)')
        exit(1)

        
def lazy_integral(f,a,b,tolerance,nmax):
    """
    Functions that integrate f from the intevral [a,b] using the dynamic simpson's rule above. 
    It basically sums over the value at each subinterval and returns the result and the number
    of counts it took for the integral to converge
        -Inputs: f (function) Function that we are interested in integrating 
                 a (float) Lower bound of the integral
                 b (float) Upper bound of the integral
                 tolerance (float) Tolerance on the error of the integral 
                 nmax (int) Maximum number of steps for the recursion. If reached, we kill
                                       it
        -Outputs: I (float) integral value 
                  count (int) Number of steps for the recursion to converge
    """

    fa = f(a)
    fb = f(b)
    guess, h, fh = simpson(f,a,b,fa,fb)
    results = np.array(dynamic_simpson(f,a,b,h,fa,fb,fh,guess,tolerance,nmax))
    count    = np.sum(results[1::2])
    I = np.sum(results[0::2])
    return I, count

In [76]:
def integrate(functions, x_min,x_max,tolerance = 1e-7,nmax=1000):
    """
    Integrates different inputed functions
        -Inputs: funcs (array of functions). These are the functions to be integrated
                 x_min (float) Lower bound of the integral 
                 x_max (float) Upper bound of the integral
                 tolerance (float) Set to default to 1e-7, this is the tolerance of the integral
                 nmax (float) Maxium number of steps given fo the integral to diverge. 
    """
    for func in functions: 
        I, count = lazy_integral(func[0],x_min,x_max,tolerance,nmax)
        error = np.abs(I - func[2](x_min)+func[2](x_max))
        print(f'The integral of {func[1]} from {x_min} to {x_max} converged to {I} in {count+1} steps, which has an error of {error}.')

        print(f'The real value is of {func[2](x_max)-func[2](x_min)}')

        print(f'We need {6*(count+1)} function evaluation with the code done in class but with this way, we get {2*(count+1)} function evaluation\n')

f = [func1(), func2(), func3()]
integrate(f,0.1,1)

The integral of 1/(1+x^2) from 0.1 to 1 converged to 0.4338330914032094 in 21.0 steps, which has an error of 0.9239321013041994.
The real value is of -0.4900990099009901
We need 126.0 function evaluation with the code done in class but with this way, we get 42.0 function evaluation

The integral of 6x^2+12x^3 from 0.1 to 1 converged to 4.997700000000001 in 1.0 steps, which has an error of 9.19837123136702e-16.
The real value is of 4.9977
We need 6.0 function evaluation with the code done in class but with this way, we get 2.0 function evaluation

The integral of 5xe^(7x) from 0.1 to 1 converged to 671.4697022060836 in 1131.0 steps, which has an error of 5.343500062773843e-07.
The real value is of 671.4697016717337
We need 6786.0 function evaluation with the code done in class but with this way, we get 2262.0 function evaluation


